In [1]:
import tmrl
import time
import matplotlib.pyplot as plts
import numpy as np
import torch
import torch.nn as nn

Hemos configurado config.json para que nos devuelva el entorno TM20LIDAR 

 "ENV": {
    "RTGYM_INTERFACE": "TM20LIDAR", 
    "WINDOW_WIDTH": 958, 
    "WINDOW_HEIGHT": 488,  
    "SLEEP_TIME_AT_RESET": 1.5, 
    "IMG_HIST_LEN": 4,
    "IMG_WIDTH": 64,  
    "IMG_HEIGHT": 64, 
    "IMG_GRAYSCALE": true, 
    "RTGYM_CONFIG": {
      "time_step_duration": 0.05,  
      "start_obs_capture": 0.04,  
      "time_step_timeout_factor": 1.0,  
      "act_buf_len": 2,  
      "benchmark": false,  
      "wait_on_done": true,  
      "ep_max_length": 2000
    },


In [ ]:
env = tmrl.get_environment()
print('Observation Space:\t', env.observation_space)
print('Action Space:\t\t', env.action_space)
print("observation space dim:", env.observation_space)

Observation Space:	 Tuple(Box(0.0, 1000.0, (1,), float32), Box(0.0, inf, (4, 19), float32), Box(-1.0, 1.0, (3,), float32), Box(-1.0, 1.0, (3,), float32))
Action Space:		 Box(-1.0, 1.0, (3,), float32)
observation space dim: None


De aquí podemos extraer que el espacio de observaciones es una tupla de 4 tipos de observación, cada una con sus características. La primera se refiere a la velocidad. La segunda, se refiere a los datos LIDAR que nos dan (tenemos las últimas 4 mediciones y 19 mediciones por cada una, ya que utiliza 19 rayos LIDAR). El espacio de observaciones nos devuelve tambien las dos últimas acciones. 
Las acciones se denotan como un espacio continuo de 3 mediciones, que son gas, frenado, y un espacio continuo entre -1.0 y 1.0 que denota la posición de las ruedas (si es 0 vamos rectos, -1.0 giramos a la izquierda y 1.0 giramos a la derecha).

In [7]:
observation_space = np.sum([np.prod(value.shape) for value in env.observation_space])
action_space = env.action_space.shape[0]
print('Número de observaciones :', observation_space)
print('Número de acciones:\t', action_space)

Número de observaciones : 83
Número de acciones:	 3


Efectivamente, tenemos 83 observaciones (1+4*19+3+3, que corresponden a velocidad, medidas LIDAR de los últimos 4 estados, y las 3 mediciones de las últimas dos acciones). 
El espacio de acciones son 3.

Ahora nos fijaremos en como TMRL computa las recompensas. En nuestro caso, la interfaz TM2020 inicializa la recompensa como una clase así:

self.reward_function = RewardFunction(reward_data_path=cfg.REWARD_PATH,
                        nb_obs_forward=cfg.REWARD_CONFIG['CHECK_FORWARD'],
                        nb_obs_backward=cfg.REWARD_CONFIG['CHECK_BACKWARD'],
                        nb_zero_rew_before_failure=cfg.REWARD_CONFIG['FAILURE_COUNTDOWN'],
                        min_nb_steps_before_failure=cfg.REWARD_CONFIG['MIN_STEPS'],
                        max_dist_from_traj=cfg.REWARD_CONFIG['MAX_STRAY'])

Y nuestro config incluye estos datos de recompensa:

"REWARD_CONFIG": {
    "END_OF_TRACK": 100.0,  
    "CONSTANT_PENALTY": 0.0, 
    "CHECK_FORWARD": 500,  
    "CHECK_BACKWARD": 10,  
    "FAILURE_COUNTDOWN": 10,  
    "MIN_STEPS": 70,  
    "MAX_STRAY": 100.0  
}

La clase RewardFunction se especifíca en tmrl.custom.tm.utils.compute_reward, y en esta se tiene un método compute_reward donde se calcula, primeramente, la distancia entre la posición actual del coche y los puntos de la trayectoria de demostración. En segundo lugar, calcula al recompensa que es el avance en la trayectoria, y si no hay avance, empieza un counter que a FAILURE_COUNTDOWN terminará el episodio. 

En nuestro caso, CHECK_FORWARD nos define cuantos puntos hacia delante buscar para encontrar coincidencias. CHECK_BACKWARD igual pero mirando hacia atrás. FAILURE_COUNTDOWN nos indica cuantos pasos sin progreso antes de terminar el episodio, siempre que hayan pasado MIN_STEPS en el episodio. MAX_STRAY nos da la distancia máxima permitida de la trayectoria.

La trayectoria es la forma en la que TMRL establece su función de recompensa. En vez de fijarse en la velocidad, se graba una trayectoria predefinida en el mapa con anterioridad al entrenamiento, y se establecen unos puntos sobre los cuales calcular la recompensa.